<h1><b>Reading Data from the STAC API</b></h1>

The Planetary Computer catalogs the datasets we host using the STAC (SpatioTemporal Asset Catalog) specification. We provide a STAC API endpoint for searching our datasets by space, time, and more. This quickstart will show you how to search for data using our STAC API and open-source Python libraries. To use our STAC API from R, see Reading data from the STAC API with R.

To get started you’ll need the pystac-client library installed.

https://planetarycomputer.microsoft.com/docs/quickstarts/reading-stac/

In [2]:
import pystac_client
import planetary_computer

In [3]:
catalog = pystac_client.Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

In [34]:
time_range = "2023-07-01/2023-12-31"
bbox = [ -105.113711, 39.567879, -104.758037, 39.809211]

search = catalog.search(collections=["landsat-c2-l2"], bbox=bbox, datetime=time_range)
items = search.item_collection()
len(items)

2

In [49]:
area_of_interest = {
    "type": "Polygon",
    "coordinates": [
        [
            [-105.113711, 39.567879],
            [-105.113711, 47.9613],
            [-121.9613, 39.809211],
            [-104.758037, 39.567879],
            [-104.758037, 39.809211],
        ]
    ],
}

time_range = "2023-07-01/2023-12-31"

search = catalog.search(
    collections=["landsat-c2-l2"], intersects=area_of_interest, datetime=time_range
)
items = search.item_collection()
len(items)

APIError: 18 validation errors for Request
body -> intersects -> type
  unexpected value; permitted: 'Point' (type=value_error.const; given=Polygon; permitted=['Point'])
body -> intersects -> coordinates
  wrong tuple length 1, expected 2 (type=value_error.tuple.length; actual_length=1; expected_length=2)
body -> intersects -> coordinates
  wrong tuple length 1, expected 3 (type=value_error.tuple.length; actual_length=1; expected_length=3)
body -> intersects -> type
  unexpected value; permitted: 'MultiPoint' (type=value_error.const; given=Polygon; permitted=['MultiPoint'])
body -> intersects -> coordinates -> 0
  wrong tuple length 5, expected 2 (type=value_error.tuple.length; actual_length=5; expected_length=2)
body -> intersects -> coordinates -> 0
  wrong tuple length 5, expected 3 (type=value_error.tuple.length; actual_length=5; expected_length=3)
body -> intersects -> type
  unexpected value; permitted: 'LineString' (type=value_error.const; given=Polygon; permitted=['LineString']

In [46]:
import geopandas
df = geopandas.GeoDataFrame.from_features(items.to_dict(), crs="epsg:4326")
df

,geometry,gsd,created,sci:doi,datetime,platform,proj:epsg,proj:shape,description,instruments,...,landsat:wrs_row,landsat:scene_id,landsat:wrs_path,landsat:wrs_type,view:sun_azimuth,landsat:correction,view:sun_elevation,landsat:cloud_cover_land,landsat:collection_number,landsat:collection_category
0,"POLYGON ((-107.04187 39.95082, -107.54639 38.2...",30,2023-07-09T09:23:02.261030Z,10.5066/P9OGBGM6,2023-07-03T17:43:23.249630Z,landsat-9,32613,"[7911, 7791]",Landsat Collection 2 Level-2,"[oli, tirs]",...,033,LC90340332023184LGN00,034,2,124.872691,L2SP,65.735034,8.95,02,T1
1,"POLYGON ((-106.60846 41.38076, -107.12748 39.6...",30,2023-07-09T09:23:01.203914Z,10.5066/P9OGBGM6,2023-07-03T17:42:59.354378Z,landsat-9,32613,"[7901, 7791]",Landsat Collection 2 Level-2,"[oli, tirs]",...,032,LC90340322023184LGN00,034,2,127.957636,L2SP,65.102823,1.97,02,T1


In [50]:
selected_item = min(items, key=lambda item: item.properties["eo:cloud_cover"])
print(selected_item)

<Item id=LC09_L2SP_034032_20230703_02_T1>


In [51]:
import rich.table

table = rich.table.Table("Asset Key", "Description")
for asset_key, asset in selected_item.assets.items():
    table.add_row(asset_key, asset.title)

table

┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Asset Key        ┃ Description                                                          ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ qa               │ Surface Temperature Quality Assessment Band                          │
│ ang              │ Angle Coefficients File                                              │
│ red              │ Red Band                                                             │
│ blue             │ Blue Band                                                            │
│ drad             │ Downwelled Radiance Band                                             │
│ emis             │ Emissivity Band                                                      │
│ emsd             │ Emissivity Standard Deviation Band                                   │
│ trad             │ Thermal Radiance Band                                                │
│ urad             │ Upwelled Radiance Band                                               │
│ atran            │ Atmospheric Transmittance Band                                       │
│ cdist            │ Cloud Distance Band                                                  │
│ green            │ Green Band                                                           │
│ nir08            │ Near Infrared Band 0.8                                               │
│ lwir11           │ Surface Temperature Band                                             │
│ swir16           │ Short-wave Infrared Band 1.6                                         │
│ swir22           │ Short-wave Infrared Band 2.2                                         │
│ coastal          │ Coastal/Aerosol Band                                                 │
│ mtl.txt          │ Product Metadata File (txt)                                          │
│ mtl.xml          │ Product Metadata File (xml)                                          │
│ mtl.json         │ Product Metadata File (json)                                         │
│ qa_pixel         │ Pixel Quality Assessment Band                                        │
│ qa_radsat        │ Radiometric Saturation and Terrain Occlusion Quality Assessment Band │
│ qa_aerosol       │ Aerosol Quality Assessment Band                                      │
│ tilejson         │ TileJSON with default rendering                                      │
│ rendered_preview │ Rendered preview                                                     │
└──────────────────┴──────────────────────────────────────────────────────────────────────┘

In [52]:
selected_item.assets["rendered_preview"].to_dict()

{'href': 'https://planetarycomputer.microsoft.com/api/data/v1/item/preview.png?collection=landsat-c2-l2&item=LC09_L2SP_034032_20230703_02_T1&assets=red&assets=green&assets=blue&color_formula=gamma+RGB+2.7%2C+saturation+1.5%2C+sigmoidal+RGB+15+0.55&format=png',
 'type': 'image/png',
 'title': 'Rendered preview',
 'rel': 'preview',
 'roles': ['overview']}

In [53]:
from IPython.display import Image

Image(url=selected_item.assets["rendered_preview"].href, width=500)

In [ ]:
import pandas as pd

search = catalog.search(
    collections=["sentinel-2-l2a"],
    bbox=[-124.2751, 45.5469, -123.9613, 45.7458],
    datetime="2020-01-01/2020-12-31",
)
items = search.get_all_items()
df = geopandas.GeoDataFrame.from_features(items.to_dict())
df["datetime"] = pd.to_datetime(df["datetime"])

ts = df.set_index("datetime").sort_index()["eo:cloud_cover"].rolling(7).mean()
ts.plot(title="eo:cloud-cover (7-scene rolling average)");